# Assignment 2 

**Team A3:** Berta Alguero, John Bergmann, Federico Colombo, Nimit Jain, Nathaniel Thomas-Copeland

## Web Scraping Linkedin

### 1. Requirements
Run the next 3 cells only if you don't have the following libraries previously installed:
- bs4
- selenium
- webdriver-manager

In [ ]:
pip install bs4

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
# Library imports

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from tqdm.auto import tqdm
import time
import string
import os
import random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

### 2. Setting the environment
Definition of fucntions that can make the later for loop shorter, and creation of the lists that will be later filed in with the scraped data.

In [ ]:
def job_details_or_none(soup_tree, index): 
    #function used to parse ul trees with identical li tags (aka lists)
    
    try:
        result = soup_tree[index].span.text.strip()
    except:
        result = None  
        
    return result

In [ ]:
def get_max_employees(employee_range): 
    #function to handle different employee sizes/displays and output them as int

    try:
        out = ""
        employees = employee_range.split("-")[1]
        
        for char in employees:
            if char in string.digits:
                out += char
            
        return int(out)
    
    except:
        out = ""
        
        for char in employee_range:
            if char in string.digits:
                out += char
                
        return int(out)

In [ ]:
def get_followers(follower_count): 
    #function to handle different follower sizes/displays and output them as int

    out = ""

    for num in follower_count:
        if num in string.digits:
            out += num

    return int(out)

In [ ]:
# Create empty lists to actually capture results when running the webscraping (by appending new elements).

job_ids = []
offer_url = []
company = []
job_title = []
location = []
state = []
posting_date = []
num_applicants = []
workspace = []
seniority = []
employment_type = []
industry = []
application_through_linkedin = []
python_required = []
promoted = []
num_employees = []
followers = []

In [ ]:
# 7 user agents(UA) to crawl with since 7 is a magic number and we don't want "python" to show up as the UA (to avoid getting blocked)

user_agents = ["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.365",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
               ]

user_agent = random.choice(user_agents)
opts = Options()
opts.add_argument(f"user-agent={user_agent}")

### 3. Web Scraping

In [ ]:
# Open a new tab and go to Linkedin

driver = webdriver.Chrome(ChromeDriverManager().install(), options = opts)

driver.maximize_window() 
driver.minimize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
driver.maximize_window() 

driver.get("https://www.linkedin.com/jobs")

# Job Query

search_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='keywords']")))
location_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='location']")))

search_field.clear()
location_field.clear()

search_input = "data scientist, barcelona"
location_input = "Barcelona, Catalonia, Spain"
time.sleep(2)

search_field.send_keys(search_input)
location_field.send_keys(location_input)
location_field.submit()
time.sleep(2)

# Log In

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-tracking-control-name='public_jobs_nav-header-signin']"))).click()

username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "username")))
password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "password")))

username_field.clear()
password_field.clear()

username = "packetlosspony@gmail.com"
password = "heqbi7-xEgvem-suhsem"
time.sleep(2)

username_field.send_keys(username)
password_field.send_keys(password)
password_field.submit()
time.sleep(2)

results = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "full-width.artdeco-entity-lockup__title.ember-view")))


# Scroll down once to define page_range

time.sleep(2)
job_soup = BeautifulSoup(driver.page_source, "html.parser")

last_page = int(job_soup.find("ul", {"class": "artdeco-pagination__pages artdeco-pagination__pages--number"}).find_all("li")[-1].text.strip())
current_page = 1
page_range = range(current_page, last_page + 1) #pythonic indexing so our tactic works :D
print(last_page)

In [ ]:
# Starts scraping

for page in tqdm(page_range, desc = "Overall Progress"):
    time.sleep(2)
    
    scroll_times = range(6)
    for x in scroll_times:
        results = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "full-width.artdeco-entity-lockup__title.ember-view")))
        driver.execute_script("return arguments[0].scrollIntoView(true);", results[-1])
        time.sleep(0.5)
    
    results = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "full-width.artdeco-entity-lockup__title.ember-view")))
    print(len(results))

    
    count = 0
    time.sleep(2)
    job_soup = BeautifulSoup(driver.page_source, "html.parser")
    job_list = job_soup.find("ul", {"class": "scaffold-layout__list-container"}).find_all("li", {"id": re.compile("^ember")})

    for job in tqdm(job_list, desc = f"Scraping page {current_page}", leave = False):
        
        job_id = job["data-occludable-job-id"]
        job_ids.append(job_id)
        time.sleep(1)
        
        # Retrieving urls 
        
        if job.find("a", {"class": "disabled ember-view job-card-container__link job-card-list__title"}) == None:
            url = None
        else:
            url = job.find("a", {"class": "disabled ember-view job-card-container__link job-card-list__title"})["href"]
            url = ("https://www.linkedin.com" + url) # Make sure we get english webpage
        offer_url.append(url)
        
        # Clicking on the next individual job offer
        
        clicker = results[count]
        time.sleep(1)
        clicker.click()
        count += 1
        time.sleep(2)
        

        # Retrieving initial information from the job offer
        
        try:
            company_name = job.find("a", {"class": "job-card-container__link job-card-container__company-name ember-view"}).text.strip()
            company.append(company_name) #extra variable needed for later
        except:
            company.append(None)

        try:
            job_title.append(job.find("a", {"class": "disabled ember-view job-card-container__link job-card-list__title"}).text.strip())
        except:
            job_title.append(None)

        try:
            location.append(job.find("li", {"class": "job-card-container__metadata-item"}).text.strip())
        except:
            location.append(None)

        try:
            workspace.append(job.find("li", {"class": "job-card-container__metadata-item job-card-container__metadata-item--workplace-type"}).text.strip())
        except:
            workspace.append(None)

        try:
            if job.find("li", {"class": "t-12 t-normal t-black--light job-card-container__footer-item"}).text.strip():
                promoted.append(True)
        except:
            promoted.append(False)

        
        # Scroll down within the inidivdual job offer page
        
        scroll_detail_tag = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="SALARY"]')))
        
        driver.execute_script("return arguments[0].scrollIntoView(true);", scroll_detail_tag)
        time.sleep(2)
        job_details = BeautifulSoup(driver.page_source, "html.parser")

        
        # Retrieving some more information from the job offer
        
        try:
            num_applicants.append(int(job_details.find("span", {"class": "jobs-unified-top-card__applicant-count"}).text.replace("applicants","").strip()))
        except:
            num_applicants.append(0) ###rare but had occasion with nothing here if "over 200" 200, except:None or 0

        try:
            posting_date.append(job_details.find("span", {"class": "jobs-unified-top-card__posted-date"}).text.strip())
        except:
            posting_date.append(None)

        
        # Some elements share the same tag and class, so we need to parse deeper
        
        job_criteria = job_details.find_all("li", {"class": "jobs-unified-top-card__job-insight"})
        
        time.sleep(1)
        for each in job_criteria:
            if each.find("li-icon") == None:
                continue
            
            # Finding same name tags using the presence of their icons
            
            # Employment Type and Seniority
            if job_details.find("li-icon", {"type" : "job"}) is not None:
                if each.find("li-icon").find("path") == job_details.find("li-icon", {"type" : "job"}).find("path"):
                    temp = each.text.strip().split("·")
                  
                    # Sometimes there is salary too making the list len 3 so we have to take the second last element for employment type
                    if len(temp) == 2:
                        employment_type.append(temp[0].strip())
                        seniority.append(temp[-1].strip())
                    elif len(temp) == 3:
                        employment_type.append(temp[-2].strip())
                        seniority.append(temp[-1].strip())
                    else:
                        employment_type.append(temp[-1].strip())
                        seniority.append(None)
            
            else:
                employment_type.append(None)
                seniority.append(None)            
                
             
            # Number of employees and Industry
            if job_details.find("li-icon", {"type" : "company"}) is not None:
                if each.find("li-icon") == job_details.find("li-icon", {"type" : "company"}):
                    temp = each.text.strip().split("·")
                    
                    if len(temp) == 1:
                        num_employees.append(int("".join(re.findall(r'\d+',temp[0].split(" ")[0].strip().split("-")[-1]))))
                        industry.append(None)
                    elif len(temp) == 2:
                        num_employees.append(int("".join(re.findall(r'\d+',temp[0].split(" ")[0].strip().split("-")[-1]))))
                        industry.append(temp[1].strip())
            else:
                num_employees.append(None)
                industry.append(None)
                
            
            # State
            if job_details.find("li-icon", {"class" : "ivm-view-attr__icon--signal-positive"}) is not None:
                if each.find("li-icon") == job_details.find("li-icon", {"class" : "ivm-view-attr__icon--signal-positive"}):
                    store = each.text.strip()
                   
                    if store in "Actively recruiting":
                        status = "On-going"
                    elif store in "Be an early applicant":
                        status = "Early applicantions"
                    else:
                        status = "Others"
            else:
                status = None
            
        state.append(status)
                    

        # Application trhough Linkeidn
        which_apply = job_details.find("span", {"class": "artdeco-button__text"}).text.strip()

        if which_apply == "Easy Apply":
            application_through_linkedin.append(True)
        else:
            application_through_linkedin.append(False)

        
        # Python Requirement
        job_description = job_details.find("div", {"id": "job-details"}).find("span").text.strip()

        if re.compile(r"\b({0})\b".format("python"), flags = re.IGNORECASE).search(job_description): #check for the actual word 'python' (case un-sensitive), not just a sequence of strings. Who knows, someone might be looking for an employee experienced with handling 'Pythonidae' snakes :D 
            python_required.append(True)
        else:
            python_required.append(False)

        # Number of Followers
        try:
            follower_num = job_details.find("div", {"class": "artdeco-entity-lockup__subtitle ember-view t-16"}).text.strip()
            followers.append(int(get_followers(follower_num)))
        except:
            followers.append(None)
        
        time.sleep(1)

    
    
    # Change onto the next page in the results
    
    current_page += 1
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[aria-label = 'Page {current_page}']"))).click()
    
    
print("Done :D") # End of loop - end of scraping.

In [ ]:
# Close the Scraping tab

driver.quit() 

### 4. Store all the data into a data frame

In [ ]:
df = pd.DataFrame({"Job Title": job_title,
                   "Company Name": company,
                   "Location": location,
                   "State": state,
                   "Posting Date": posting_date,
                   "Offer URL": offer_url,
                   "Number of Applicants": num_applicants,
                   "Promoted": promoted,
                   "Workspace": workspace,
                   "Seniority": seniority,
                   "Employment Type": employment_type,
                   "Industry": industry,
                   "Python Required": python_required,
                   "Application through Linkedin": application_through_linkedin,
                   "Number of Employees": num_employees,
                   "Followers": followers
                   })

In [ ]:
df.head()